# Classification Plugins


**AutoPrognosis** provides a set of default predicton plugins and can be extended with any number of other plugins.

In this tutorial, we will focus on the __classification__ tasks.

### Plugins 101

Every **AutoPrognosis plugin** must implement the **`Plugin`** interface provided by `autoprognosis/plugins/core/base_plugin.py`.

Each **AutoPrognosis prediction plugin** must implement the **`PredictionPlugin`** interface provided by `autoprognosis/plugins/prediction/base.py`

__Warning__ : If a plugin doesn't override all the abstract methods, it won't be loaded by the library.




__API__ : Every prediction plugin must implement the following methods:
- `name()` - a static method that returns the name of the plugin. e.g., neural_nets, perceptron, etc.

- `subtype()` - a static method that returns the plugin's subtype. e.g., "classification", "survival_analysis" etc. It will be used for filtering the plugin in the optimization process.
    
- `hyperparameter_space()` - a static method that returns the hyperparameters that can be tuned during the optimization. The method will return a list of `skopt.space.Dimension` derived objects.

### Setup

In [ ]:
# stdlib
import sys
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tabulate

# third party
from IPython.display import HTML, display
from sklearn import metrics
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tqdm import tqdm

if not sys.warnoptions:
    warnings.simplefilter("ignore")

### Loading the Classification plugins

Make sure that you have installed AutoPrognosis in your workspace.

You can do that by running `pip install .` in the root of the repository.

In [ ]:
# autoprognosis absolute
from autoprognosis.plugins.prediction.classifiers import ClassifierPlugin, Classifiers

classifiers = Classifiers()

### List the existing plugins

In [ ]:
classifiers.list_available()

### List the existing plugins

Now we should see the new plugins loaded.

In [ ]:
# autoprognosis absolute
from autoprognosis.plugins.prediction.classifiers import Classifiers

classifiers = Classifiers()

classifiers.list_available()

## Benchmarks

We test the prediction plugins using the [Wisconsin Breast Cancer dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)).

### Loading the data

In [ ]:
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Duration benchmarks

__About__ : This step measures the fit_predict duration for each plugin on the dataset. The times are reported in milliseconds.

In [ ]:
duration = []

plugins = classifiers.list_available()

for plugin in tqdm(plugins):
    plugin_duration = [plugin]
    ctx = classifiers.get(plugin)

    start = time.time() * 1000
    ctx.fit_predict(X, y)

    plugin_duration.append(round(time.time() * 1000 - start, 4))

    duration.append(plugin_duration)

### Duration(ms) results

In [ ]:
display(
    HTML(
        tabulate.tabulate(duration, headers=["Plugin", "Duration(ms)"], tablefmt="html")
    )
)

### Prediction performance

__Steps__
 - We train each prediction plugin on the dataset.
 - We report the AUCROC using cross-validation.

In [ ]:
# autoprognosis absolute
from autoprognosis.utils.tester import evaluate_estimator


def get_metrics(plugin):
    score = evaluate_estimator(plugin, X, y)["str"]
    return score["aucroc"]


plugins = classifiers.list_available()

metrics_headers = ["Plugin", "AUCROC"]
test_score = []


for plugin in plugins:
    fproc = classifiers.get(plugin)

    score = get_metrics(fproc)

    test_score.append([plugin, score])

In [ ]:
display(HTML(tabulate.tabulate(test_score, headers=metrics_headers, tablefmt="html")))

# Congratulations!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards Machine learning and AI for medicine, you can do so in the following ways!

### Star AutoPrognosis on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the tools we're building.

- [Star AutoPrognosis](https://github.com/vanderschaarlab/autoprognosis)
- [Star HyperImpute](https://github.com/vanderschaarlab/hyperimpute)
